In [1]:
import bs4
import json
import requests
from bs4 import BeautifulSoup

link = "https://secure.meetcontrol.com/divemeets/system/index.php"


In [2]:
html = requests.get(link).text
soup = BeautifulSoup(html, "html.parser")
tabs = (
    soup.find("div", {"id": "dm_menu_centered"})
    .find("ul")
    .find_all("li", recursive=False)
)
tab_dict = {tab.find("a", recursive=False).text: tab for tab in tabs}


In [3]:
seen_orgs = dict()

meets = (
    tab_dict["Meets"]
    .find("ul", recursive=False)
    .find("li", recursive=False)
    .find_all("a")[1:]
)
upcoming_meets = {m.text.strip(): m["href"] for m in meets}


In [4]:
pmeets = (
    tab_dict["Meets"]
    .find("ul", recursive=False)
    .find_all("li", recursive=False)[1]
    .find_all("a")[1:]
)
past_meets = dict()
last_key = None
for a in pmeets:
    if a["href"] == "#":
        last_key = a.text
        past_meets[last_key] = dict()
    else:
        past_meets[last_key][a.text.strip()] = a["href"]


In [5]:
# with open("upcoming_meets.json", "w") as f:
#     json.dump(upcoming_meets, f)

# with open("past_meets.json", "w") as f:
#     json.dump(past_meets, f)


In [6]:
parse_link ="https://secure.meetcontrol.com/divemeets/system/profile.php?number=56961" 
soup = BeautifulSoup(requests.get(parse_link).text, "html.parser")
result = soup.find("td").text
name_split = result.split("Name: ")[1].split("City/State: ")
name = name_split[0]
city_split = name_split[1].split("Country: ")
city = city_split[0]
country_split = city_split[1].split("Gender: ")
country = country_split[0]
age_split = country_split[1].split("FINA Age: ")
gender_split = age_split[0].split("Age: ")
gender = gender_split[0]
age = gender_split[1]
fina_age_split = age_split[1].split("High School Graduation: ")
fina_age = fina_age_split[0]
hs_grad_split = fina_age_split[1].split("DiveMeets #: ")
hs_grad = hs_grad_split[0]
dm_num = hs_grad_split[1].split("Diving:")[0]

In [57]:
# Logan
# parse_link = "https://secure.meetcontrol.com/divemeets/system/profile.php?number=56961" 
# Beck
# parse_link = "https://secure.meetcontrol.com/divemeets/system/profile.php?number=16241"
# Jesus
# parse_link = "https://secure.meetcontrol.com/divemeets/system/profile.php?number=56797"
# Stephanie
parse_link = "https://secure.meetcontrol.com/divemeets/system/profile.php?number=20617"

soup = BeautifulSoup(requests.get(parse_link).text, "html.parser")

In [60]:
items = []
headers = {"Diving:", "Coaching:", "Judging:"}
found_header = False
parse_list = []
diver_list = []

for i in soup.td.contents:
    if i == ')' or (isinstance(i, bs4.element.NavigableString) and i.strip() == ""):
        continue
    if not found_header and not isinstance(i, bs4.element.NavigableString) and i.text.strip() in headers:
        found_header = True
    elif not found_header:
        continue
    if isinstance(i, bs4.element.NavigableString):
        items.append(i.strip())
    elif i.name == 'strong':
        items.append(i.text.strip())
    elif i.name == 'a':
        items.append((i.text.strip(), i['href']))
    elif i.name == 'center':
        parse_list = items
        items = []
    elif i.name == 'table':
        break

diver_list = items
print(parse_list)
print(diver_list)
items = parse_list

i = 0
# Looks for "Diving:" line to start parsing teams instead of personal info
# found_diving = False
# new_items = {}
orgs = {}
orgName = ""
while i < len(items):
    if items[i] == "Diving:" or items[i] == "Coaching:" or items[i] == "Judging:":
        orgName = items[i][:-1]
    elif orgName == "Diving":
        if orgName not in orgs:
            orgs[orgName] = dict()
        coach, link = items[i+3]
        link = link[:7] + link[8:]
        link = 'https://secure.meetcontrol.com/divemeets/system/' + link
        org = {"Team":items[i+1], "Coach":coach, "Link": link}
        orgs["Diving"][items[i][:-1]] = org
        i += 3
    elif orgName == "Coaching":
        if orgName not in orgs:
            orgs[orgName] = dict()
        orgs["Coaching"][items[i]] = items[i+1][:-2]
        i += 2
    elif orgName == "Judging":
        if orgName not in orgs:
            orgs[orgName] = []
        orgs[orgName].append(items[i])
    i += 1
# new_items = orgs
if diver_list != []:
    orgs["Diver List"] = diver_list
orgs


['Coaching:', 'Amateur Athletic Union (AAU):', 'Dominion Dive Club (', ('Coach Profile', 'profilec.php?number=20617'), 'USA Diving:', 'Dominion Dive Club (', ('Coach Profile', 'profilec.php?number=20617'), 'Judging:', 'Amateur Athletic Union (AAU)', 'USA Diving']
[('Grosz, Madison', 'profile.php?number=43244'), ('Hansen, Kenneth V', 'profile.php?number=16072'), ('Palo, Brett', 'profile.php?number=16036'), ('Perreault, Joseph', 'profile.php?number=30060'), ('Powell, Max', 'profile.php?number=33610'), ('Warren, Matthew K', 'profile.php?number=37560'), ('Donnell, Minh', 'profile.php?number=43205'), ('Duncan, Samuel B', 'profile.php?number=40135'), ('Eisenberg, Michayla L', 'profile.php?number=47307'), ('Flory, Maxwell L', 'profile.php?number=23733'), ('Flory, Samuel A', 'profile.php?number=41815'), ('Gould, Mory E', 'profile.php?number=43888'), ('Green, Tanya M', 'profile.php?number=80844'), ('Joyce, Ellie C', 'profile.php?number=50039'), ('Miller, Liam M', 'profile.php?number=41743'), ('

{'Coaching': {'Amateur Athletic Union (AAU):': 'Dominion Dive Club',
  'USA Diving:': 'Dominion Dive Club'},
 'Judging': ['Amateur Athletic Union (AAU)', 'USA Diving'],
 'Diver List': [('Grosz, Madison', 'profile.php?number=43244'),
  ('Hansen, Kenneth V', 'profile.php?number=16072'),
  ('Palo, Brett', 'profile.php?number=16036'),
  ('Perreault, Joseph', 'profile.php?number=30060'),
  ('Powell, Max', 'profile.php?number=33610'),
  ('Warren, Matthew K', 'profile.php?number=37560'),
  ('Donnell, Minh', 'profile.php?number=43205'),
  ('Duncan, Samuel B', 'profile.php?number=40135'),
  ('Eisenberg, Michayla L', 'profile.php?number=47307'),
  ('Flory, Maxwell L', 'profile.php?number=23733'),
  ('Flory, Samuel A', 'profile.php?number=41815'),
  ('Gould, Mory E', 'profile.php?number=43888'),
  ('Green, Tanya M', 'profile.php?number=80844'),
  ('Joyce, Ellie C', 'profile.php?number=50039'),
  ('Miller, Liam M', 'profile.php?number=41743'),
  ('Reed, Dylan D', 'profile.php?number=43209'),
  ('S